In [1]:
import kagglehub

path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/skin-cancer-mnist-ham10000


In [2]:
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import numpy as np
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
from glob import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Recall, Accuracy
import tensorflow as tf

2025-12-14 00:50:44.203420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765673444.397322      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765673444.450579      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
df = pd.read_csv('../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')


In [5]:
df["dx"].value_counts()

dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64

In [6]:
df= df.drop(columns=["lesion_id","age","sex","localization","dx_type"])


In [7]:
df["dx"].unique()

array(['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec'], dtype=object)

In [8]:
label_mapping = {
    'mel': 1,
    'bcc': 1,
    'akiec': 1,
    'nv': 0,
    'bkl': 0,
    'vasc': 0,
    'df': 0
}

df['dx'] = df['dx'].replace(label_mapping)

/tmp/ipykernel_47/506550335.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['dx'] = df['dx'].replace(label_mapping)


In [9]:
df_0 = df[df['dx'] == 0]
df_1 = df[df['dx'] == 1]

df_0_down = df_0.sample(2000, random_state=42)

df = pd.concat([df_0_down, df_1]).sample(frac=1, random_state=42)

In [10]:
df["dx"].value_counts()

dx
0    2000
1    1954
Name: count, dtype: int64

In [11]:
from glob import glob
import os

image_paths = glob("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_*/*.jpg")

path_dict = {os.path.basename(p).replace(".jpg", ""): p for p in image_paths}

df['path'] = df['image_id'].map(path_dict)

In [12]:
import numpy as np
from PIL import Image

IMG_SIZE = 236

X = []
y = []

for _, row in df.iterrows():
    img = Image.open(row['path']).convert("RGB")
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img = np.array(img) / 255.0                    
    
    X.append(img)
    y.append(row['dx'])

X = np.array(X)
y = np.array(y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [14]:
cnn = tf.keras.models.Sequential()

In [15]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[236,236, 3]))
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1765673617.447083      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [17]:
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [18]:
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [19]:
cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [20]:
cnn.add(tf.keras.layers.Dropout(0.25))

In [21]:
cnn.add(tf.keras.layers.MaxPooling2D())
cnn.add(tf.keras.layers.Flatten())   # <-- Obligatoire avant Dense
cnn.add(tf.keras.layers.Dense(128, activation='relu'))

In [22]:
cnn.add(tf.keras.layers.Dropout(0.4)) #To avoid overfitting

In [23]:
cnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))


In [24]:
cnn.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy',"recall"])

In [25]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 236, 236, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 234, 234, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 117, 117, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 117, 117, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 115, 115, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 57, 57, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 57, 57, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 55, 55, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 27, 27, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 27, 27, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 25, 25, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 12, 12, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 10, 10, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 5, 5, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 5, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,974,625 (18.98 MB)

 Trainable params: 4,974,625 (18.98 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:

cnn.fit(
    X, y,
    validation_split=0.1,
    epochs=20,
   
)


Epoch 1/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.7500 - loss: 0.4983 - recall: 0.8365 - val_accuracy: 0.7753 - val_loss: 0.4779 - val_recall: 0.9552
Epoch 2/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.7620 - loss: 0.4784 - recall: 0.8654 - val_accuracy: 0.7677 - val_loss: 0.4612 - val_recall: 0.8159
Epoch 3/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.7606 - loss: 0.4614 - recall: 0.8304 - val_accuracy: 0.7803 - val_loss: 0.4889 - val_recall: 0.8607
Epoch 4/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.7628 - loss: 0.4751 - recall: 0.8555 - val_accuracy: 0.7576 - val_loss: 0.4584 - val_recall: 0.8060
Epoch 5/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.7756 - loss: 0.4500 - recall: 0.8482 - val_accuracy: 0.7904 - val_loss: 0.4602 - val_recall: 0.8955
Epoch 6/20
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.7410 - loss: 0.4939 - recall: 0.8309 - val_accuracy: 0.7551 - val_loss: 0.4659 - val_recall:

In [31]:
import joblib 
joblib.dump(cnn,"mymodel.pkl")

['mymodel.pkl']

In [27]:
import numpy as np
np.unique(y)


array([0, 1])

In [29]:
import numpy as np

print("NaN X_train :", np.isnan(X).sum())
print("Inf X_train :", np.isinf(X).sum())



NaN X_train : 0
Inf X_train : 0


In [30]:
model.evaluate(X_test, y_test, verbose=1)

NameError: name 'model' is not defined

In [ ]:
model.predict(X_test, verbose=2)